In [1]:
import ee
ee.Initialize()
import matplotlib
from matplotlib.pylab import plt
import numpy as np
import pandas as pd
import seaborn as sns
#print(s2c.getInfo())
%matplotlib inline

In [56]:
#S2 cloud score and shadow removal parameters
cloudThresh =20 #lower value more cloud filtered
irSumThresh =0.35 #Sum of IR bands to include as shadows within TDOM and the shadow shift method (lower number masks out less)
dilatePixels = 2 #Pixels to dilate around clouds
contractPixels = 1 #Pixels to reduce cloud mask and dark shadows by to reduce inclusion of single-pixel comission errors

In [2]:
#Time frame for temporal composites
start = '2018-05-01'
end = '2018-09-30'

In [3]:
#Define features
random_samples = 10
extent = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_boundary')
felled = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_felled_only').map(lambda f : f.set('polytype', 'young'))
young = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_young_trees').map(lambda f : f.set('polytype', 'young'))
mature = ee.FeatureCollection('users/tomw_ee/Inputs/kielder_mature_trees').map(lambda f : f.set('polytype', 'young'))
#randomFc = ee.FeatureCollection.randomPoints(young, random_samples).map(lambda f : f.set('polytype', 'young'))
#randomFelled = ee.FeatureCollection.randomPoints(felled, random_samples).map(lambda f : f.set('polytype', 'felled'))
#randomMature = ee.FeatureCollection.randomPoints(mature, random_samples).map(lambda f : f.set('polytype', 'mature'))
#randomFc = randomFc.merge(randomMature).merge(randomFelled)
randomFc = felled.merge(young).merge(mature)

In [26]:
#Bands are divided by 10000 and renamed for cloud score
def s2_bands(img):
    t = img.select([ 'B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9','B10', 'B11','B12']).divide(10000)
    t = t.addBands(img.select(['QA60']))
    out = t.copyProperties(img).copyProperties(img,['system:time_start'])
    return out

In [14]:
#Masking function for S2 cloud using QA band
def maskS2clouds(image):
    qa = image.select('QA60')
     #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 2**10
    cirrusBitMask = 2**11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    #.and(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask)

In [15]:
#Masking function for S2 cloud using cloud score 
def sentinelCloudScore(img):
  # Compute several indicators of cloudyness and take the minimum of them.
    score = ee.Image(1)  
  # Clouds are reasonably bright in the blue and cirrus bands.
    score = score.min(rescale(img, 'img.blue', [0.1, 0.5]))
    score = score.min(rescale(img, 'img.cb', [0.1, 0.3]))
    score = score.min(rescale(img, 'img.cb + img.cirrus', [0.15, 0.2])) 
  # Clouds are reasonably bright in all visible bands.
    score = score.min(rescale(img, 'img.red + img.green + img.blue', [0.2, 0.8]))
  # Clouds are moist
    ndmi = img.normalizedDifference(['nir','swir1'])
    score=score.min(rescale(ndmi, 'img', [-0.1, 0.1]))
  # However, clouds are not snow.
    ndsi = img.normalizedDifference(['green', 'swir1'])
    score=score.min(rescale(ndsi, 'img', [0.8, 0.6]))
    score = score.multiply(100).byte()
    return img.addBands(score.rename('cloudScore'))

In [46]:
# TDOM2 function for removing cloud shadow
def simpleTDOM2(coll):
    shadowSumBands = ['nir','swir1']
    irSumThresh = 0.4
    zShadowThresh = -1.2
    #Get some pixel-wise stats for the time series
    irStdDev = coll.select(shadowSumBands).reduce(ee.Reducer.stdDev())
    irMean = coll.select(shadowSumBands).mean()
    bandNames = ee.Image(coll.first()).bandNames()
    #Mask out dark outliers using mask dark function
    def mask_dark(img):
        z = img.select(shadowSumBands).subtract(irMean).divide(irStdDev)
        irSum = img.select(shadowSumBands).reduce(ee.Reducer.sum())
        m = z.lt(zShadowThresh).reduce(ee.Reducer.sum()).eq(2).And(irSum.lt(irSumThresh)).Not()
        return img.updateMask(img.mask().And(m))
    coll = coll.map(mask_dark)
    return coll.select(bandNames)

In [51]:
def rescale(img, exp, thresholds):
    return img.expression(exp, {img: img}).subtract(thresholds[0]).divide(thresholds[1] - thresholds[0])

In [95]:
#Get S2 collection
pnt = ee.Geometry.Point([-3.3616017971660312,55.9070462518514])
polyFeature = ee.Feature(geometry)

s2c = ee.ImageCollection('COPERNICUS/S2')\
.filterDate('2018-09-01', '2018-09-30')\
.filterBounds(geometry)\
.map(s2_bands)\
.select(['QA60', 'B1','B2','B3','B4','B5','B6','B7','B8','B8A', 'B9','B10', 'B11','B12'],['QA60','cb', 'blue', 'green', 'red', 're1','re2','re3','nir', 'nir2', 'waterVapor', 'cirrus','swir1', 'swir2'])


#Apply TDOM
s2c = simpleTDOM2(s2c)

In [102]:
#Test s2 cloud shadow masks on one image
n = s2c.size()
colList = s2c.toList(n)
img1 = maskS2clouds(ee.Image(colList.get(0)))
img1 = sentinelCloudScore(img)
img1 = img.updateMask(img.select(['cloudScore']).gt(cloudThresh).focal_min(contractPixels).focal_max(dilatePixels).Not())


In [110]:
fromList = ee.FeatureCollection(pnt)
val = img1.reduceRegions(fromList,ee.Reducer.first(),10)
val.getInfo()

TypeError: keys must be a string

In [98]:
#Define image collections
s2c = ee.ImageCollection('COPERNICUS/S2').filterDate(start, end).filterBounds(extent).map(maskS2clouds)
s1c = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(extent).filterDate(start, end).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

### 1. Distribution of pixel values for different land uses
* Reduce image collection to a median composite over the time frame
* Sample the pixel values at all random points for mature, young, felled land uses

In [99]:
#Median composite creation
s1_med = s1c.median()
s2_med = s2c.median()
s2_med= s2_med.addBands(s2_med.normalizedDifference(['B8', 'B4']).rename('ndvi'))

In [100]:
s1_red = s1_med.reduceRegions(randomFc, ee.Reducer.first(),10)
s2_red = s1_med.reduceRegions(randomFc, ee.Reducer.first(),10)

In [107]:
def export_results(coll, output_file_name):
    task=ee.batch.Export.table.toDrive(
            collection=coll,
            folder = 'Pixel_Values',
            fileFormat='CSV',
            fileNamePrefix=output_file_name)
    task.start()

In [9]:
export_results(s1_red, 's1_results_median_luse')
export_results(s2_red, 's2_results_median_luse')

## 2. Temporal Data September 2016 to September 2018

Only done for sentinel 1 due to cloud problem with trying to do the same with sentinel 2

In [32]:
start = '2018-09-15'
end = '2018-09-30'

In [33]:
s2c = ee.ImageCollection('COPERNICUS/S2').filterDate(start, end).filterBounds(extent).map(maskS2clouds)
s1c = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(extent).filterDate(start, end).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).\
filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

In [49]:
def temporal_values(img):
    return img.reduceRegions(randomFc, ee.Reducer.first(),10).map(lambda f : f.set('img_date', img.date().format('dd-MM-yyyy')))
    

In [50]:
temporal_s1 = s1c.map(temporal_values).flatten()
temporal_s2 = s2c.map(temporal_values).flatten()

In [51]:
export_results(temporal_s1,'temporal_test2')
expport_results(temporal_s2, 'temporal_test2')